## Raw Agent

Slightly improved based on https://learn.deeplearning.ai/courses/ai-agents-in-langgraph

In [ ]:
%pip install -r requirements.txt

In [181]:
from openai import OpenAI
from dotenv import load_dotenv
from functools import partial
import re
_ = load_dotenv()

In [182]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")


In [183]:
actions = [
    {
        "name": calculate,
        "usage": "e.g. calculate: 4 * 7 / 3",
        "desc": "Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary"
    },
    {
        "name": average_dog_weight,
        "usage": "e.g. average_dog_weight: Scottish Terrier",
        "desc": "Returns the average weight of a dog breed"
    }
]

In [184]:
prompt_template = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

{actions}
Here is a session template:

Question: [Users question]?
Thought: [Your thought to use an action]
Action: [selected action]: [action parameter]
PAUSE

You will be called again with the result of the action you selected::

Observation: [action result]

You then output:

Answer: [Your answer]
"""

In [185]:
class ReActAgent:
    def __init__(self, prompt_template, actions, openai_model = "gpt-4o-mini"):
        actions_in_prompt = ""
        for a in actions:
            actions_in_prompt += f"{a['name'].__name__}\n{a['usage']}\n{a['desc']}\n\n"

        self.system = prompt_template.format(actions = actions_in_prompt)
        self.messages = []
        self.messages.append({"role": "system", "content": self.system})
        self.chat = partial(OpenAI().chat.completions.create, model=openai_model, temperature=0)

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        completion = self.chat(messages=self.messages)
        out = completion.choices[0].message.content
        self.messages.append({"role": "assistant", "content": out})
        return out

In [186]:
agent = ReActAgent(prompt_template, actions)

**Checking the system prompt**

In [ ]:
print(agent.system)

**Testing Thought, Action, PAUSE**

In [ ]:
out = agent("How much does a Toy Poodle weigh?")
print(out)


**Testing action call, Observation**

In [ ]:
obs = f"Observation: {average_dog_weight("Toy Poodle")}"
obs

In [ ]:
out = agent(obs)
print(out)

**List of messages**

In [ ]:
agent.messages

## Looping

**Parse action in the output**

In [150]:
action_re = re.compile(r"Action: (\w+): (.*)") 

In [161]:
ex_output = """
Question: How much does a Toy Poodle weigh?
Thought: I need to find the average weight of a Toy Poodle, so I will use the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE
"""
r = action_re.search(ex_output)

In [156]:
if(r):
    action, param = r.groups()

**Calling the selected action**

In [ ]:
# Find the action with the name "average_dog_weight"
action = next((a for a in actions if a['name'].__name__ == action), None)

# Call the action if it exists
if action:
    result = action['name'](param)
    print(result)

**Building all together**

In [177]:
def ask_agent(agent, question, max_turn=5):
    out = ""
    next_input_to_agent = question
    for i in range(max_turn):
        out = agent(next_input_to_agent)
        r = action_re.search(out)
        if not(r):
            # if there's no action to call then we are done
            break
        print("--- Agent thought process ------")
        print(out)
        print("--------------------------------")
        
        action, param = r.groups()
        action = next((a for a in actions if a['name'].__name__ == action), None)
        if not(action):
            raise Exception(f"Unknown action: {action}")
        next_input_to_agent = f"Observation: {action['name'](param)})"
    return out
    

In [ ]:
agent = ReActAgent(prompt_template, actions)
answer = ask_agent(agent, "How much does a Toy Poodle weigh?")
print(answer)


In [ ]:
agent = ReActAgent(prompt_template, actions)
answer = ask_agent(agent, "I have 2 dogs, a border collie and a scottish terrier.\
                 What is their combined weight")
print(answer)